# This notebook is to prepare a dataset for training the ANNABELL model.
The dataset is derived from the SQuAD database.  Each Question and Answer pair was used to prompt a LLM to provide a declarative statement.


In [17]:
import pandas as pd
import re
import subprocess
from collections import Counter
from generate_declarative_sentences import load_squad_dataset
from unidecode import unidecode
import tarfile

## To be useable for training, the ANNABELL model,  the examples need to follow the specific format below:
* Uppercase letters are used only for first letter of proper nouns – e.g. Chris, London, Big Ben
* Questions start with a question mark – e.g. "how old are you"
* Words with a suffix are split in the form base –suffix.  e.g. animals -> animal –s, writing \t *> write \t *ing
    * Apart from the above exceptions the following rules apply:
    * every character must be lowercase
    * No punctuation
    * No Special Characters
    * No Whitespace between lines
    * Lines can be prefixed with # to insert comments
    * If .ph is used, the entire phrase in the exact format must be input

In [2]:
def remove_quotes(text):
    # remove " form the text
    cleaned_text = text.replace('"', '')
    return cleaned_text

In [3]:
def remove_quotes_from_file(filepath):
    with open(filepath, "r") as file:
        train_data = file.readlines()
        train_data_cleaned = [remove_quotes(line) for line in train_data]

    cleaned_filepath = filepath.replace(".tsv", "_cleaned.tsv")
    with open(cleaned_filepath, "w") as cleaned_file:
        cleaned_file.writelines(train_data_cleaned)
    print(f"Cleaned data saved to {cleaned_filepath}")
    return cleaned_filepath

In [5]:
# Load the SQuAD dataset
ds = load_squad_dataset()

# Load the datafile into a pandas DataFrame
train_filepath = remove_quotes_from_file("/Volumes/X9 Pro/datasets/declarative_sentences_train_gemma3:4b_20250617_201822.tsv")
validation_filepath = remove_quotes_from_file("/Volumes/X9 Pro/datasets/declarative_sentences_validation_gemma3:4b_20250618_200853.tsv")
train_df = pd.read_csv(train_filepath, sep="\t")
validation_df = pd.read_csv(validation_filepath, sep="\t")
#remove rows with null values in any columns
train_df = train_df.dropna()
validation_df = validation_df.dropna()
print(train_df.info())
print(validation_df.info())

Cleaned data saved to /Volumes/X9 Pro/datasets/declarative_sentences_train_gemma3:4b_20250617_201822_cleaned.tsv
Cleaned data saved to /Volumes/X9 Pro/datasets/declarative_sentences_validation_gemma3:4b_20250618_200853_cleaned.tsv
<class 'pandas.core.frame.DataFrame'>
Index: 85419 entries, 0 to 85483
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 85419 non-null  object
 1   title              85419 non-null  object
 2   question           85419 non-null  object
 3   answer             85419 non-null  object
 4   response_question  85419 non-null  object
 5   response_answer    85419 non-null  object
 6   statement          85419 non-null  object
dtypes: object(7)
memory usage: 5.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id     

In [6]:
#move the ? from the end of each question to the start
def move_question_mark_to_start(question, add_if_missing=True):

    if question.strip().endswith("?"):
        edited_question = "? " + question[:-1]
    else:
        if add_if_missing:
            edited_question = "? " + question
        else:
            #raise an exception if the question does not end with a ?
            raise ValueError(f"Question does not end with a question mark: {question}")

    return edited_question

In [7]:
def replace_decimal_in_matched_string(matched_string):
    #callable function to support the regex
    number_str = matched_string.group(0)
    return number_str.replace('.', ' point ')

In [8]:
def convert_decimal_point_to_word(a_string):
    #Replace "." with 'point' if it is part of a number.

    # Regex to find numbers with a decimal point:
    # \d+\.\d+  matches numbers like 1.23
    # \d+\.     matches numbers like 1. (dot at the end after digits)
    # \.\d+     matches numbers like .5 (dot at the beginning before digits)
    # The order matters to match \d+\.\d+ before \d+\. or \.\d+ for overlapping cases.
    pattern = r'\d+\.\d+|\d+\.|\.\d+'
    return re.sub(pattern, replace_decimal_in_matched_string, a_string)

In [9]:
def remove_accents(text):
    #Convert accented characters to unaccented ones
    text_unaccented = unidecode(text)
    return text_unaccented

In [10]:
#remove all special characters except question marks and hyphen form the statements
def remove_special_characters(text):
    """
    Removes special characters from a string, keeping alphanumeric characters and spaces.
    """
    # Keep only alphanumeric characters and spaces
    cleaned_text = re.sub(r'[^A-Za-z0-9\s?-]+', '', text)
    return cleaned_text

In [11]:
def filter_by_max_words(the_df, max_words=10):
    #returnes a new dataframe filtered such that each question, answer and statement has less than 11 words

    filtered_df = the_df[the_df.apply(lambda row: len(row["question"].split()) <= max_words and len(row["answer"][0].split()) <= max_words and len(row["statement"].split()) <= max_words , axis=1)]
    return filtered_df

In [12]:
def clean_text(a_series, is_question):
    #takes a dataframe series and applies reformatting
    if is_question:
        a_series = a_series.apply(move_question_mark_to_start)
    for function_name in (convert_decimal_point_to_word, remove_accents, remove_special_characters):
        a_series = a_series.apply(function_name)
    return a_series

In [13]:
filtered_train_df = filter_by_max_words(train_df, max_words=20)
filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
filtered_train_df["statement"] = clean_text(filtered_train_df["statement"], False)
filtered_train_df

/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_3838/4088644761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_3838/4088644761.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_3838/4

,id,title,question,answer,response_question,response_answer,statement
0,5733be284776f41900661182,University_of_Notre_Dame,To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,? to whom did the virgin mary allegedly appear...,saint Bernadette Soubirous,the virgin mary allegedly appeared in 1858 in ...
1,5733be284776f4190066117f,University_of_Notre_Dame,What is in front of the Notre Dame Main Building?,a copper statue of Christ,? what is in front of the Notre Dame Main Buil...,a copper statue of Christ,the Notre Dame Main Building is in front of a ...
2,5733be284776f41900661180,University_of_Notre_Dame,The Basilica of the Sacred heart at Notre Dame...,the Main Building,? the Basilica of the Sacred heart at Notre Da...,the Main Building,the Basilica of the Sacred heart at Notre Dame...
3,5733be284776f41900661181,University_of_Notre_Dame,What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,? what is the grotto at notre dame,a Marian place of prayer and reflection,the grotto at notre dame is a Marian place of ...
4,5733be284776f4190066117e,University_of_Notre_Dame,What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,? what sits on top of the Main Building at Not...,a golden statue of the Virgin Mary,the Main Building has a golden statue of the V...
...,...,...,...,...,...,...,...
85478,5732a8a6328d981900601fed,Geological_history_of_Earth,Which current resulted in the cooling of Antar...,the Antarctic Circumpolar Current,? which current resulted in the cooling of Ant...,the Antarctic Circumpolar Current,the Antarctic Circumpolar Current resulted in ...
85480,5732ac1fcc179a14009dabe7,Geological_history_of_Earth,Which continent was India colliding with in th...,Asia,? Which continent was India colliding with in ...,asia,india was colliding with asia in the Miocene
85481,5732ac1fcc179a14009dabe8,Geological_history_of_Earth,When Africa was colliding with Eurasia which s...,The Tethys Seaway,? when Africa was colliding with Eurasia which...,the Tethys Seaway,Africa was colliding with Eurasia the Tethys S...
85482,5732ac1fcc179a14009dabe9,Geological_history_of_Earth,Between what period of time did the Tethys dis...,19 and 12 Ma,? between what period of time did the tethys d...,19 and 12 ma,the tethys disappeared between the period of 1...


In [19]:
#write a file that can be used to train ANNABELL
filepath_minus_extension = "training/filtered_train_data"
output_filepath = filepath_minus_extension + ".txt"
with open(output_filepath, "w") as output_file:
    for statement in filtered_train_df["statement"]:
        output_file.write(statement + "\n")
#check that the file looks correct
with open(output_filepath, "r") as input_file:
    lines = input_file.readlines()
print(lines[:5])
#compress the file to tar.xz format
with tarfile.open(filepath_minus_extension + ".tar.xz", "w:xz") as tar:
    tar.add(output_filepath, arcname="output_filepath.txt")
print(f"Compressed file created: {filepath_minus_extension}.tar.xz")

['the virgin mary allegedly appeared in 1858 in lourdes france to saint Bernadette Soubirous\n', 'the Notre Dame Main Building is in front of a copper statue of Christ\n', 'the Basilica of the Sacred heart at Notre Dame is beside the Main Building\n', 'the grotto at notre dame is a Marian place of prayer and reflection\n', 'the Main Building has a golden statue of the Virgin Mary on top of it\n']
Compressed file created: training/filtered_train_data.tar.xz


In [24]:
#produce a summary of a dataset by splits
def dataset_summary(a_dataset):
    for split in ds.keys():
        ds_split=ds[split].to_pandas()
        print("summary of " + split + " split")
        print(ds_split.info())
        titles=ds_split["title"]
        print("number of titles: " + str(len(set(titles))))
        print((set(titles)))
        bag_of_titles = Counter((titles))
        print("titles with most numerous examples: " + str((bag_of_titles.most_common(20))) +"\n")

dataset_summary(ds)

In [25]:
#Evaluate the results
# simplest case - take 1 example of a question, answer and statement, ask annabell the question, record the answer, check the answer against the statement
# use the id of the sample to link the 3 items together
# or just iterate though a dataframe containing the orginal data plus the answer.

filtered_train_df

,id,title,question,answer,response_question,response_answer,statement
1,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? in what city is the united nations based,new york,the united nations is based in New York
3,56ce304daab44d1400b88511,New_York_City,What American city welcomes the largest number...,New York,? What American city welcomes the largest numb...,new york,new york welcomes the largest number of legal ...
6,56ce3124aab44d1400b8852a,New_York_City,How many boroughs comprise New York City?,five,? How many boroughs comprise New York City,five,New York City comprise -s five borough -s
11,56cf9d81234ae51400d9be1b,New_York_City,How man boroughs does New York City contain?,five,? How man boroughs does New York City contain,five,New York City contains five borough -s
17,56ce31baaab44d1400b8853b,New_York_City,What nation founded New Amsterdam?,the Dutch Republic,? What nation founded New Amsterdam,the Dutch Republic,the Dutch Republic founded New Amsterdam
...,...,...,...,...,...,...,...
790,56d11b4a17492d1400aab994,New_York_City,The Queensboro Bridge utilized what type of co...,cantilever,? the Queensboro Bridge utilized what type of ...,cantilever,the Queensboro Bridge utilized a cantilever co...
796,56d11e7b17492d1400aab9d3,New_York_City,How long is Newtown Creek in kilometers?,6,? How long is Newtown Creek in kilometers,6,newtown creek is 6 kilometers long
798,56d11e7b17492d1400aab9d5,New_York_City,What notable accidental fossil fuel discharge ...,the Greenpoint oil spill,? what notable accidental fossil fuel discharg...,the Greenpoint oil spill,the Greenpoint oil spill occurred at Newtown C...
799,56d11eb317492d1400aab9d9,New_York_City,What type of government does New York City have?,mayor-council,? what type of government does New York City have,mayor-council,new york city has a mayor-council government


In [ ]:
# Define the command to execute.
# Replace './your_script.sh' with the actual path to your script
# and add any arguments it might need.
command = ['./your_script.sh', 'arg1', 'arg2']

# Execute the command
# 'capture_output=True' stores stdout and stderr
# 'text=True' decodes stdout and stderr as text (Python 3.7+)
# 'check=True' will raise a CalledProcessError if the command returns a non-zero exit code
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)

    # Print the standard output and standard error
    print("STDOUT:")
    print(result.stdout)
    if result.stderr:
        print("STDERR:")
        print(result.stderr)

except FileNotFoundError:
    print(f"Error: The script {command[0]} was not found.")
except subprocess.CalledProcessError as e:
    print(f"Error executing script: {command[0]}")
    print(f"Return code: {e.returncode}")
    print("STDOUT:")
    print(e.stdout)
    print("STDERR:")
    print(e.stderr)

In [15]:
import subprocess

command_str = """
(echo .load crossvalidation/round1/links/links_people_body_skills.dat && \\
 echo .logfile log_nyc_train.txt && \\
 echo .logfile off && \\
 echo .save crossvalidation/round1/links/links_people_body_skills_nyc.dat && \\
 echo .q) | annabell
"""
try:
    result = subprocess.run(
        command_str,
        capture_output=True,
        text=True,
        check=True,
        shell=True,
        cwd="/home/chris/PycharmProjects/dataset"
    )

    print("Output of command:")
    print(result.stdout)
    if result.stderr:
        print("STDERR:")
        print(result.stderr)

except subprocess.CalledProcessError as e:
    print(f"Error executing command: {e}")
    print(e.stderr)

Output of command:
Threads max number: 22
Free parameters:
seed: 12345
CW_W: 1
InPhB_W: 3
WkPhB_W: 1
WkEqWG_W: 3
WkEqGoalWG_W: 5
GoalPhEqGoalWG_W: 1
OutPhB_W: 1
MaxDynamicBias_W: 500
Enter command: .load crossvalidation/round1/links/links_people_body_skills.dat
Enter command: .logfile log_nyc_train.txt
Enter command: .logfile off
Enter command: .save crossvalidation/round1/links/links_people_body_skills_nyc.dat
Enter command: 


In [14]:
import subprocess

# Define the command as a single string
# Ensure 'annabell' is in PATH or use an absolute/relative path.
# If 'annabell' is in '/home/chris/annabell' and not in PATH,
# you might need 'cd /home/chris/annabell && (echo ...) | ./annabell'
# or use the 'cwd' argument in subprocess.run.

command_str = """
(echo .load crossvalidation/round1/links/links_people_body_skills.dat && \\
 echo .logfile log_nyc_train.txt && \\
 echo .logfile off && \\
 echo .save crossvalidation/round1/links/links_people_body_skills_nyc.dat && \\
 echo .q) | annabell
"""
# Assuming annabell is at /home/chris/annabell/annabell and needs full path
# Or, if annabell is in PATH, just 'annabell' is fine.
# Using '&&' between echo commands for robustness if one were to fail, though for echo it's less critical.

try:
    # Execute the command using shell=True
    result = subprocess.run(
        command_str,
        capture_output=True,
        text=True,
        check=True,
        shell=True,
        cwd="/home/chris/PycharmProjects/dataset" # Optional: set current working directory if annabell expects to be run from its own directory
                                    # and paths inside the echo commands are relative to it.
                                    # If paths in echo are absolute, cwd might be less critical for them,
                                    # but still important for finding 'annabell' if not using full path for it.
    )

    print("Output of command:")
    print(result.stdout)
    if result.stderr:
        print("STDERR:")
        print(result.stderr)

except FileNotFoundError:
    # This can happen if 'annabell' or another part of the command isn't found
    # or if the shell itself isn't found (less likely).
    print(f"Error: Command or a component not found. Check paths and executables.")
    print(f"Attempted command: {command_str}")
except subprocess.CalledProcessError as e:
    print(f"Error executing shell command:")
    print(f"Return code: {e.returncode}")
    print(f"Command: {e.cmd}")
    print("STDOUT:")
    print(e.stdout)
    print("STDERR:")
    print(e.stderr)

Output of command:
Threads max number: 22
Free parameters:
seed: 12345
CW_W: 1
InPhB_W: 3
WkPhB_W: 1
WkEqWG_W: 3
WkEqGoalWG_W: 5
GoalPhEqGoalWG_W: 1
OutPhB_W: 1
MaxDynamicBias_W: 500
Enter command: .load crossvalidation/round1/links/links_people_body_skills.dat
Enter command: .logfile log_nyc_train.txt
Enter command: .logfile off
Enter command: .save crossvalidation/round1/links/links_people_body_skills_nyc.dat
Enter command: 
